In [ ]:
!pip install imblearn

# Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler as ROS
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Read data and split into x and y

In [ ]:
train_data = pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/train.csv")
test_data = pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/test.csv")

In [ ]:
y_train = train_data["target"]
x_train = train_data.drop(["target", "ID_code"], axis=1)
x_test = test_data.drop("ID_code", axis=1)

# Data exploration

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(y_train.unique())
print(x_test.shape)

In [ ]:
t1 = y_train[y_train == 1].index
t0 = y_train[y_train == 0].index
print("t1 data percentage: ", len(t1)/len(y_train)*100)
print("t0 data percentage: ", len(t0)/len(y_train)*100)

Pretty Imbalanced data!

# Prediction

In [ ]:
x_resampled, y_resampled = ROS().fit_resample(x_train, y_train)
scaler = StandardScaler()
x_train_sc = scaler.fit_transform(x_resampled)
x_test_sc = scaler.transform(x_test)
prediction = np.zeros(len(x_test))
skf = StratifiedKFold(n_splits=10, shuffle=True)
for train_index, test_index in skf.split(x_train_sc, y_resampled):
    X_train, X_test = x_train_sc[train_index], x_train_sc[test_index]
    Y_train, Y_test = y_resampled[train_index], y_resampled[test_index]
    clf = LGBMClassifier().fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print("roc auc accuracy: ", metrics.roc_auc_score(y_pred, Y_test))
    
    prediction += clf.predict(x_test_sc)/skf.n_splits

# Submission

In [ ]:
test_data["target"]=prediction
test_data[["ID_code", "target"]].to_csv("submission.csv", index=False)